<a href="https://colab.research.google.com/github/yuva23/Ds-python/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow nltk numpy pandas scikit-learn


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk

nltk.download('stopwords')
nltk.download('punkt')

# Sample data (You can replace this with a real dataset)
data = {
    'text': [
        'I am not happy with the product',
        'The service was excellent',
        'The delivery was late',
        'Great experience overall',
        'I want to return my order'
    ],
    'sentiment': ['negative', 'positive', 'negative', 'positive', 'negative']
}
df = pd.DataFrame(data)

# Text preprocessing
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = ' '.join(text)
    return text

df['text'] = df['text'].apply(preprocess_text)

# Encode sentiments
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000
max_len = 50
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


In [4]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           500000    
                                                                 
 spatial_dropout1d (Spatial  (None, 50, 100)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 580501 (2.21 MB)
Trainable params: 580501 (2.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5
1/1 [==============================] - 6s 6s/step - loss: 0.6919 - accu

In [5]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Accuracy: {accuracy*100}')


1/1 [==============================] - 0s 34ms/step - loss: 0.9469 - accuracy: 0.0000e+00
Accuracy: 0.0


In [ ]:
import random

responses = {
    'positive': ['Thank you for your feedback!', 'We are glad you enjoyed our service!'],
    'negative': ['We are sorry for the inconvenience.', 'Your feedback is valuable to us.']
}

def predict_sentiment(text):
    text = preprocess_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len)
    pred = model.predict(padded)
    sentiment = 'positive' if pred >= 0.5 else 'negative'
    return sentiment

def get_response(text):
    sentiment = predict_sentiment(text)
    return random.choice(responses[sentiment])

while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    response = get_response(user_input)
    print(f"Bot: {response}")


You: I want to return product 
1/1 [==============================] - 0s 29ms/step
Bot: Your feedback is valuable to us.
You: i am not satisfied 
1/1 [==============================] - 0s 25ms/step
Bot: Your feedback is valuable to us.
You: great experience over all 
1/1 [==============================] - 0s 27ms/step
Bot: We are sorry for the inconvenience.
You: i want to return my order
1/1 [==============================] - 0s 36ms/step
Bot: We are sorry for the inconvenience.
You: i liked the experience 
1/1 [==============================] - 0s 30ms/step
Bot: Your feedback is valuable to us.
